In [6]:
import os
import sys
import re
from pathlib import Path


In [7]:
# Create some data structures and variables for our program
filepath = Path("./budget_data.csv")

budget = {}
profit_loss_total = 0
profit_loss_high = []
profit_loss_low = []

# Open the file "budget_data.csv"
with open(filepath, mode = "r") as f:

# Create an iterable "i" that we will use for error references and a container for them:
    i = 0
    error_list = []
    
    
#Start the main loop
    for line in f:
        i += 1 
        if i > 0:
             
            # Check for a specific concantenation of 3 letters for month, 4 digits for year, a comma,
            # Then everything after that (which SHOULD only be numbers). 
            budget_item_match = re.search(r"(\w\w\w-\d\d\d\d),(.+)", line)

            #This is simple enough but creates one issue. in the second group (.+), we just grab everything after the comma
            #Some numbers have a negative sign, some do not. We need to make certain that our second item is a number (imagine someone typed in "n/a" 
            # or accidently entered a "Q" instead of "1")
            if budget_item_match:
                #Set a variable equal to our P/l as a string object, so that we can iterate through it real quick and make sure it fits the right format
                check_pl_data = budget_item_match.group(2)
                #Here we loop through each "ch" character in the string check_data. If the character is a negative sign or a digit, there is no issue. 
                for ch in check_pl_data:
                    if ch == "-":
                        pass
                    elif ch.isdigit():
                        pass
                    else:
                        print(f"Error on line # {i}: {line} -- {ch} is causing an error")
                        print(f"File must include 1 header row in first line, date in first column (MON-YEAR, ex: JAN-2001), and profit/loss in 2nd colummn without $ (ex: -189000)")
                        print("Please remove or reformat line, save the file, and retry")
                        sys.exit(1)
                #Now we want to start tracking some P/L data as it"s coming in:
                if profit_loss_total == 0:
                    profit_loss_total = int(budget_item_match.group(2))
                    profit_loss_high = [budget_item_match.group(1) ,int(budget_item_match.group(2))]
                    profit_loss_low = [budget_item_match.group(1) ,int(budget_item_match.group(2))]
                    new_key = 1
                    budget[new_key] = (budget_item_match.group(1),int(budget_item_match.group(2)))
                else:
                    #At this point, I want to force our budget{} keys to be sequential regardless of how many blank lines there are:
                    new_key += 1 
                    budget[new_key] = (budget_item_match.group(1),int(budget_item_match.group(2)))
                    profit_loss_total = profit_loss_total + int(budget_item_match.group(2))
                    #Tracks the most recent high mark for monthly p/l
                    if profit_loss_high[-1] <= int(budget_item_match.group(2)):
                        profit_loss_high = [budget_item_match.group(1) ,int(budget_item_match.group(2))]
                    #Tracks the most recent low mark for monthly p/l
                    elif profit_loss_low[-1] >= int(budget_item_match.group(2)):
                        profit_loss_low = [budget_item_match.group(1) ,int(budget_item_match.group(2))]
            # First we check for a line we don"t know what to do with:    
            if not budget_item_match:
                #Next we skip the first row, where the header is expected anyway and we don"t need those
                if i == 1:
                    pass
                # Next we check for an empty line, or an empty line with whitespaces (which is why using line == "\n" would not comprehensively solve the issue)
                elif len(line.strip()) == 0:
                    print(f"Line # {i} is empty")
                #Finally if there is still an issue, we error out and print the line with an issue, formatting instructions, and directions to user
                else:
                    print(f"Error on line # {i}: {line}")
                    print(f"File must include 1 header row in FIRST LINE, date in first column (MON-YEAR, ex: JAN-2001), and profit/loss in 2nd colummn without $ (ex: -189000)")
                    print("Please remove or reformat line, save the file, and retry")
                    sys.exit(1)

#Now we need to calculate the proper average. A dictionary isn"t sorted inherently, but since we iterated new_key, all the key-value pairs match up with the 
#monthly progression of the data. So step 1) grab all the values and turn them into a list:
budget_items_object = budget.values()
budget_tuples_list = list(budget_items_object)
p_l_list = []
deltas = []
#step 2) loop through all the tuples and grab the profit from each month, creating a list of all the SEQUENTIAL CHANGES:
for v in range(len(budget_tuples_list)):
    budget_tup = budget_tuples_list[v]
    p_l_list.append(budget_tup[1])
#step 3) build a list of all the month-to-month deltas in p/l:
    if v >= 1:
        deltas.append(p_l_list[v] - p_l_list[v-1])
#step 4) calculate average change:
average_deltas = (sum(deltas)/len(deltas))

print("Financial Analysis")
print("----------------------------")
print(f"Total Months: {new_key}")
print(f"Total: {profit_loss_total}")
print(f"Average Change: ${round(average_deltas,2)}")
print(f"Greatest Increase in Profits: {profit_loss_high[0]} (${profit_loss_high[1]})")
print(f"Greatest Decrease in Profits: {profit_loss_low[0]} (${profit_loss_low[1]})")


Financial Analysis
----------------------------
Total Months: 86
Total: 38382578
Average Change: $-2315.12
Greatest Increase in Profits: Feb-2012 ($1170593)
Greatest Decrease in Profits: Sep-2013 ($-1196225)


In [8]:
summarypath = Path("summary.txt")
with open(summarypath, "w") as summary_f:
    summary_f.write("Financial Analysis\n----------------------------\n")
    summary_f.write(f"Total Months: {new_key}\n")
    summary_f.write(f"Total: {profit_loss_total}\n")
    summary_f.write(f"Average Change: ${round(average_deltas,2)}\n")
    summary_f.write(f"Greatest Increase in Profits: {profit_loss_high[0]} (${profit_loss_high[1]})\n")
    summary_f.write(f"Greatest Decrease in Profits: {profit_loss_low[0]} (${profit_loss_low[1]})\n")
summary_f.close()